## New Pre-processing methods and decomposition models

Useful methods for pre-processing the galaxy object to perform the dynamical decomposition:

 - Center funtion: `preproc.Centralizer()`
 - Star align function : `preproc.Aligner(r_cut=30)`
 - Stellar half mass radius: `preproc.Cutter(num_radii=3)`
 - Potential energy: `preproc.Potentializer(backend="POTENTIAL_BACKENDS")`

In [2]:
import galaxychop as gchop

gal = gchop.io.read_hdf5("gal394242.h5")
gal

<Galaxy stars=32067, dark_matter=21156, gas=4061, potential=True>

The packages provides two function to validate if a galaxy is centered and aligned:

In [ ]:
center = gchop.preproc.Centralizer()
align = gchop.preproc.Aligner(r_cut=30) 

print("Centered:", center.checker(gal))
print("Aligned:", align.checker(gal))

Centered: False
Aligned: False


First of all, to perform the dynamical decomposition is necesary center the position of all galaxy particles respect to the position of the lowest potential particle.

In [ ]:
gal = center.transform(gal)  

<Galaxy stars=32067, dark_matter=21156, gas=4061, potential=True>

Secondly, the position particles are rotatated so that the total angular moment of the stars particles coincides
with the z-axis.

Optionally, only stars particles within a cutting radius `(r_cut)` can be used to calculate the rotation matrix.

In [ ]:
gal= align.transform(gal)

In [ ]:
print("Centered:", center.checker(gal))
print("Aligned:", align.checker(gal))

Centered: True
Aligned: True
Centered & Aligned: True


Optionally one could perform the calculus of the stellar half mass radius

In [ ]:
cutter = gchop.preproc.Cutter(num_radii=3)
gal = cutter.transform(gal)

In [ ]:
print("Cutted:", cutter.checker(gal))

r_sh = gchop.preproc.smr_crop.get_radius_half_mass(gal, particle='stars')
print("Stellar half-mass radius:" + f' {r_sh:.2f} kpc\n')

Cutted: True


And the calculation of the potential energy, using this backends to perform the calculation

`POTENTIAL_BACKENDS = {
    "fortran",
    "grispy",
    "numpy"
}`

In [4]:
pot = gchop.preproc.Potentializer("fortran")

gal=pot.transform(gal)

CREATED POTENCIALIZER WITH BACKEND  fortran


## Decomposition

To perform the dynamical decomposition the package provides several model:

    - JThreshold
    - JHistogram
    - JEHistogram
    - KMeans
    - GaussianMixture
    - AutoGaussianMixture
    
The procedure to find to which stellar component each star particle belongs is exactly the same for all models:

First instantiate the decomposer

In [7]:
decomposer = gchop.models.JThreshold()
decomposer

JThreshold(cbins=(0.05, 0.005), eps_cut=0.6)

Decompose the galaxy into components

In [ ]:
gal_decomposed = decomposer.decompose(gal)
components = gal_decomposed.components

<Components length=57284, labels={'Spheroid', nan, 'Disk'}, probabilities=False, lmap=True>

In [9]:
components.labels

array([ 0.,  0.,  0., ..., nan, nan, nan])

In [10]:
components.ptypes

array(['stars', 'stars', 'stars', ..., 'gas', 'gas', 'gas'], dtype='<U11')

In [11]:
components.to_dataframe()